In [26]:
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace, date_format
from pyspark.sql.types import IntegerType, DoubleType, FloatType

In [27]:
dir_hdfs = "hdfs://namenode:9000/tmp/amd/locatel0311"

In [28]:
df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)
df.printSchema()

root
 |-- id_folio: string (nullable = true)
 |-- fecha_solicitud: string (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tipo_de_entrada: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- fecha_concluido: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- 0311_colonia_registro: string (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [54]:
datos = df.select(['fecha_solicitud','hora_solicitud','tema_solicitud','sexo','edad','estatus','alcaldia','colonia_datos','latitud','longitud', 'codigo_postal_solicitud'])

In [55]:
datos = datos.withColumn("fecha_solicitud", to_date(col("fecha_solicitud"), "yyyy-MM-dd")) \
        .withColumn("hora_solicitud", date_format(col("fecha_solicitud"), "HH:MM:SS")) \
        .withColumn("edad", col("edad").cast(FloatType())) \
        .withColumn("latitud", col("latitud").cast(DoubleType())) \
        .withColumn("longitud", col("longitud").cast(DoubleType())) 

datos.printSchema()

root
 |-- fecha_solicitud: date (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)



In [56]:
from pyspark.sql.functions import avg, max, min, count
dfg = datos.groupBy("alcaldia")\
.agg(avg("edad").alias("edad_promedio"),
     max("edad").alias("edad_maxima"), 
     min("edad").alias("edad_minima"), 
     count("*").alias("no_registros")
    )

In [57]:
dfg.orderBy("alcaldia", ascending=False).show(100, truncate=False)

+----------------------+------------------+-----------+-----------+------------+
|alcaldia              |edad_promedio     |edad_maxima|edad_minima|no_registros|
+----------------------+------------------+-----------+-----------+------------+
|XOCHIMILCO            |44.02038006058937 |100.0      |0.0        |50073       |
|VENUSTIANO CARRANZA   |34.12961033548656 |93.0       |0.0        |16713       |
|TLALPAN               |39.53971856559237 |98.0       |0.0        |54937       |
|TLAHUAC               |27.59472115793955 |88.0       |0.0        |17358       |
|Otro                  |NULL              |NULL       |NULL       |1           |
|NA                    |46.65257871998707 |100.0      |0.0        |856239      |
|MILPA ALTA            |33.32932330827067 |88.0       |0.0        |2879        |
|MIGUEL HIDALGO        |4.992536800110057 |96.0       |0.0        |97139       |
|LA MAGDALENA CONTRERAS|34.029847908745246|99.0       |0.0        |8593        |
|IZTAPALAPA            |34.5

In [58]:
from pyspark.sql.functions import when

datos = datos.withColumn("alcaldia", when(datos.alcaldia == "IZTACALCO ", "IZTACALCO").otherwise(datos.alcaldia))

In [59]:
sepomex_df = spark.read.option("header", "true").option("delimiter", "|").option("inferSchema", "true")\
    .csv("hdfs://namenode:9000/tmp/amd/sepomex/")

sepomex_df.printSchema()

root
 |-- d_codigo: integer (nullable = true)
 |-- d_asenta: string (nullable = true)
 |-- d_tipo_asenta: string (nullable = true)
 |-- D_mnpio: string (nullable = true)
 |-- d_estado: string (nullable = true)
 |-- d_ciudad: string (nullable = true)
 |-- d_CP: integer (nullable = true)
 |-- c_estado: integer (nullable = true)
 |-- c_oficina: integer (nullable = true)
 |-- c_CP: string (nullable = true)
 |-- c_tipo_asenta: integer (nullable = true)
 |-- c_mnpio: integer (nullable = true)
 |-- id_asenta_cpcons: integer (nullable = true)
 |-- d_zona: string (nullable = true)
 |-- c_cve_ciudad: integer (nullable = true)



In [60]:
datos.groupBy("alcaldia")\
.agg(avg("edad").alias("edad_promedio"),
max("edad").alias("edad_maxima"),
min("edad").alias("edad_minima"),
count("*").alias("no_registros")
).orderBy("alcaldia", ascending=False).show(100, truncate=False)

+----------------------+------------------+-----------+-----------+------------+
|alcaldia              |edad_promedio     |edad_maxima|edad_minima|no_registros|
+----------------------+------------------+-----------+-----------+------------+
|XOCHIMILCO            |44.02038006058937 |100.0      |0.0        |50073       |
|VENUSTIANO CARRANZA   |34.12961033548656 |93.0       |0.0        |16713       |
|TLALPAN               |39.53971856559237 |98.0       |0.0        |54937       |
|TLAHUAC               |27.59472115793955 |88.0       |0.0        |17358       |
|Otro                  |NULL              |NULL       |NULL       |1           |
|NA                    |46.65257871998707 |100.0      |0.0        |856239      |
|MILPA ALTA            |33.32932330827067 |88.0       |0.0        |2879        |
|MIGUEL HIDALGO        |4.992536800110057 |96.0       |0.0        |97139       |
|LA MAGDALENA CONTRERAS|34.029847908745246|99.0       |0.0        |8593        |
|IZTAPALAPA            |34.5

In [61]:
sepomex_df.show(truncate=False)

+--------+------------------------------------+-------------+--------------+----------------+----------------+----+--------+---------+----+-------------+-------+----------------+------+------------+
|d_codigo|d_asenta                            |d_tipo_asenta|D_mnpio       |d_estado        |d_ciudad        |d_CP|c_estado|c_oficina|c_CP|c_tipo_asenta|c_mnpio|id_asenta_cpcons|d_zona|c_cve_ciudad|
+--------+------------------------------------+-------------+--------------+----------------+----------------+----+--------+---------+----+-------------+-------+----------------+------+------------+
|1000    |San Ángel                           |Colonia      |Álvaro Obregón|Ciudad de México|Ciudad de México|1001|9       |1001     |NULL|9            |10     |1               |Urbano|1           |
|1010    |Los Alpes                           |Colonia      |Álvaro Obregón|Ciudad de México|Ciudad de México|1001|9       |1001     |NULL|9            |10     |5               |Urbano|1           |
|1020

In [62]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
 
schema = StructType([
    StructField("d_codigo", StringType(), True),
    StructField("d_asenta", StringType(), True),
    StructField("d_tipo_asenta", StringType(), True),
    StructField("D_mnpio", StringType(), True),
    StructField("d_estado", StringType(), True),
    StructField("d_ciudad", StringType(), True),
    StructField("d_CP", StringType(), True),
    StructField("c_estado", StringType(), True),
    StructField("c_oficina", StringType(), True),
    StructField("c_CP", StringType(), True),
    StructField("c_tipo_asenta", StringType(), True),
    StructField("c_mnpio", StringType(), True),
    StructField("id_asenta_cpcons", StringType(), True),
    StructField("d_zona", StringType(), True),
    StructField("c_cve_ciudad", StringType(), True)
])

In [63]:
sepomex_df = spark.read.option("header", "true").option("delimiter", "|").option("skipRows", "1")\
    .option("encoding", "UTF-8").schema(schema).csv("hdfs://namenode:9000/tmp/amd/sepomex/")

sepomex_df.show(truncate = False)

+--------+------------------------------------+-------------+--------------+----------------+----------------+-----+--------+---------+----+-------------+-------+----------------+------+------------+
|d_codigo|d_asenta                            |d_tipo_asenta|D_mnpio       |d_estado        |d_ciudad        |d_CP |c_estado|c_oficina|c_CP|c_tipo_asenta|c_mnpio|id_asenta_cpcons|d_zona|c_cve_ciudad|
+--------+------------------------------------+-------------+--------------+----------------+----------------+-----+--------+---------+----+-------------+-------+----------------+------+------------+
|01000   |San Ángel                           |Colonia      |Álvaro Obregón|Ciudad de México|Ciudad de México|01001|09      |01001    |NULL|09           |010    |0001            |Urbano|01          |
|01010   |Los Alpes                           |Colonia      |Álvaro Obregón|Ciudad de México|Ciudad de México|01001|09      |01001    |NULL|09           |010    |0005            |Urbano|01          |


In [64]:
dfcp = sepomex_df.select(['d_codigo', 'D_mnpio', 'd_estado'])

dfcp.count()

155922

In [65]:
dfcp.show(truncate = False)
datos.printSchema()

+--------+--------------+----------------+
|d_codigo|D_mnpio       |d_estado        |
+--------+--------------+----------------+
|01000   |Álvaro Obregón|Ciudad de México|
|01010   |Álvaro Obregón|Ciudad de México|
|01020   |Álvaro Obregón|Ciudad de México|
|01030   |Álvaro Obregón|Ciudad de México|
|01030   |Álvaro Obregón|Ciudad de México|
|01040   |Álvaro Obregón|Ciudad de México|
|01049   |Álvaro Obregón|Ciudad de México|
|01050   |Álvaro Obregón|Ciudad de México|
|01060   |Álvaro Obregón|Ciudad de México|
|01060   |Álvaro Obregón|Ciudad de México|
|01070   |Álvaro Obregón|Ciudad de México|
|01080   |Álvaro Obregón|Ciudad de México|
|01089   |Álvaro Obregón|Ciudad de México|
|01090   |Álvaro Obregón|Ciudad de México|
|01090   |Álvaro Obregón|Ciudad de México|
|01090   |Álvaro Obregón|Ciudad de México|
|01100   |Álvaro Obregón|Ciudad de México|
|01109   |Álvaro Obregón|Ciudad de México|
|01110   |Álvaro Obregón|Ciudad de México|
|01110   |Álvaro Obregón|Ciudad de México|
+--------+-

In [66]:
from pyspark.sql.functions import col, coalesce

datos_mapeados = datos.join(dfcp.select("d_codigo", "D_mnpio"), \
                            datos["codigo_postal_solicitud"] == dfcp["d_codigo"], "left")

datos_mapeados.printSchema()

root
 |-- fecha_solicitud: date (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- d_codigo: string (nullable = true)
 |-- D_mnpio: string (nullable = true)



In [68]:
datos_mapeados.select(['codigo_postal_solicitud', 'alcaldia', 'd_codigo', 'D_mnpio']).show(100, truncate=False)

+-----------------------+----------+--------+---------------------+
|codigo_postal_solicitud|alcaldia  |d_codigo|D_mnpio              |
+-----------------------+----------+--------+---------------------+
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NULL                 |
|NA                     |NA        |NULL    |NUL

In [69]:
datos_mapeados.withColumn("alcaldia", coalesce(col("D_mnpio"), col("alcaldia")))

DataFrame[fecha_solicitud: date, hora_solicitud: string, tema_solicitud: string, sexo: string, edad: float, estatus: string, alcaldia: string, colonia_datos: string, latitud: double, longitud: double, codigo_postal_solicitud: string, d_codigo: string, D_mnpio: string]

In [70]:
datos_mapeados.select(["edad", "alcaldia", "codigo_postal_solicitud", "d_codigo"]).show(100, truncate = False)

+----+----------+-----------------------+--------+
|edad|alcaldia  |codigo_postal_solicitud|d_codigo|
+----+----------+-----------------------+--------+
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA                     |NULL    |
|NULL|NA        |NA            